In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib  # is a library often used for saving and loading Python objects efficiently, especially large ones
import re   # re is the regular expressions module in Python, used for searching, matching, and manipulating strings using regex patterns


# Load the data  -- >   Two columns   1.text (email body/subject)  , 2. lable( 0 = ham/not_spam , 1 = spam)

data = pd.DataFrame({

    "text": [
        "limited time offer!!! click on this link to win a prize",
        "Hey, We can reschedule our meeting tomorrow  ?",
        "URGENT : Your account has been suspended please verify now.",
        "here are the slides from the class --see you !.",
        "you won a lottery , claim your cash reward now!"
        
        
    ],

    "lable" : [1,0,1,0,1]
    
})



# light text cleaner

def clean_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"(https?://\S+| www\.S+)", "URL", s)
    s = re.sub(r"[^a-z0-9@.\s]", "", s)
    s = re.sub(r"\s+", "", s).strip()
    
    return s



def apply_cleaning(df : pd.DataFrame ,text_col: str ="text") -> pd.DataFrame:
    out = df.copy()
    out[text_col] = out[text_col].astype(str).map(clean_text)
    return out


# train/val the dataset




